In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = 'https://www.basketball-reference.com/'
browser.visit(url)

In [4]:
player_list = ['Giannis Antetokounmpo','James Harden','Russell Westbrook','Stephen Curry',
              'Kevin Durant','LeBron James','Derrick Rose','Kobe Bryant','Dirk Nowitzki','Steve Nash',
              'Kevin Garnett','Tim Duncan','Allen Iverson',"Shaquille O'Neal",'Karl Malone','Michael Jordan',
              'David Robinson','Hakeem Olajuwon','Charles Barkley','Magic Johnson']

In [5]:
player_stats = pd.DataFrame()


for player in player_list:
    try:
        # fill out player name
        browser.find_by_css('.ac-input').first.fill(player)
        # click search for player
        browser.find_by_value('Search').first.click()
        # click link to player for players that come up in search
        try:
            browser.click_link_by_partial_text(player + " (")
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            time.sleep(3)
            
            # finding the totals table
            table_totals = soup.find('div', {'id':'div_totals'})
            #create dataframe from totals
            df1 = pd.read_html(str(table_totals))[0]
            #finding the advanced table data
            table_advanced = soup.find('div', {'id' : 'div_advanced'})
            # create dataframe from advanced tables
            df2 = pd.read_html(str(table_advanced))[0]
            # create column with player name
            df1['Player'] = player
#             df1 = df1.reset_index(drop=True)
#             df2 = df2.reset_index(drop=True)
            df = pd.merge(df1, df2, on= "Season", how = "left")

#             player_stats.append(df1, ignore_index = True)
            player_stats = pd.concat([player_stats, df], ignore_index=True)
        except:
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            time.sleep(3)
            # finding the totals table
            table_totals = soup.find('div', {'id':'div_totals'})
            #create dataframe from totals
            df1 = pd.read_html(str(table_totals))[0]
            #finding the advanced table data
            table_advanced = soup.find('div', {'id' : 'div_advanced'})
            # create dataframe from advanced tables
            df2 = pd.read_html(str(table_advanced))[0]
            # create column with player name
            df1['Player'] = player
#             df1 = df1.reset_index(drop=True)
#             df2 = df2.reset_index(drop=True)
            df = pd.merge(df1, df2, on= "Season", how = "left")

#             player_stats = player_stats.append(df1, ignore_index = True)
            player_stats = pd.concat([player_stats, df], ignore_index=True)
    except:
        pass
    
browser.quit()

/Users/dotcal/anaconda3/envs/PythonDataanalytics/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/dotcal/anaconda3/envs/PythonDataanalytics/lib/python3.6/site-packages/ipykernel_launcher.py:52: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [6]:
player_stats.to_csv('MVP_stats.csv', encoding='utf-8')